# 1. Introduction: Team ChabaCrunch

This notebook presents an organized workflow for processing the TouchBistro x UW dataset, which consists of two key files: `venues.csv` and `bills.csv`. Our process begins with data cleaning and preprocessing, during which we address missing values and outliers. Next, we implement a machine learning model to predict missing venue “concept” entries. With these refinements in place, we perform exploratory data analysis (EDA) to gain insights into customer spending behaviors, highlighting the potential for data-driven decision-making in the restaurant industry.

# 2. Imports & Data Loading

In [ ]:
import os
import zipfile
import gdown
import pandas as pd
import numpy as np
import gc  # For garbage collection

def optimize_numeric(df):
    """
    Downcast numeric columns to reduce memory usage.
    """
    for col in df.select_dtypes(include=[np.number]).columns:
        if pd.api.types.is_integer_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif pd.api.types.is_float_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# Define the Google Drive file ID
file_id = "1T6EcNXe_bnFhVevxLFXWn_obh1Xl-01F"

# Define local filenames
zip_path = "/content/dataset.zip"
extract_path = "/content/dataset"

# Download the ZIP file from Google Drive
gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_path, quiet=False)

# Verify if the file is a valid ZIP archive before extracting
if zipfile.is_zipfile(zip_path):
    print("Valid ZIP file. Extracting...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
else:
    print("Error: Downloaded file is not a ZIP. Check Google Drive permissions or file link.")
    exit()

# Identify the correct directory inside the extracted ZIP
extracted_folders = os.listdir(extract_path)
print("Extracted Contents:", extracted_folders)

# Locate the actual data folder (CxCData)
data_folder = os.path.join(extract_path, "CxCData")
if not os.path.exists(data_folder):
    print("Error: Expected folder 'CxCData' not found. Check the ZIP structure.")
    exit()

# Update paths to the correct location
venues_path = os.path.join(data_folder, "venues.csv")
bills_path = os.path.join(data_folder, "bills.csv")

# Load CSVs
venues_df = pd.read_csv(venues_path)
bills_df = pd.read_csv(bills_path)

# Immediately downcast numeric columns in both DataFrames
venues_df = optimize_numeric(venues_df)
bills_df = optimize_numeric(bills_df)

print("Venues DataFrame Shape:", venues_df.shape)
print("Bills DataFrame Shape:", bills_df.shape)

Downloading...
From (original): https://drive.google.com/uc?id=1T6EcNXe_bnFhVevxLFXWn_obh1Xl-01F
From (redirected): https://drive.google.com/uc?id=1T6EcNXe_bnFhVevxLFXWn_obh1Xl-01F&confirm=t&uuid=d6bb2a90-8413-4ae3-87e3-5aa313949fa4
To: /content/dataset.zip
100%|██████████| 842M/842M [00:06<00:00, 127MB/s]


Valid ZIP file. Extracting...
Extracted Contents: ['__MACOSX', 'CxCData']
Venues DataFrame Shape: (601, 5)
Bills DataFrame Shape: (8954786, 20)


# 3. Initial Inspection

Next, we take a quick look at the basic structure of our datasets. We also convert relevant fields to the correct data types.

In [ ]:
# Quick inspection
# venues_df.info()
# print(venues_df.head())
# bills_df.info()
# print(bills_df.head())

date_cols = ["bill_paid_at_local", "order_seated_at_local", "order_closed_at_local", "business_date"]
for col in date_cols:
    bills_df[col] = pd.to_datetime(bills_df[col], errors='coerce')

for col in date_cols:
    print(f"{col} dtype: {bills_df[col].dtype}")

bill_paid_at_local dtype: datetime64[ns]
order_seated_at_local dtype: datetime64[ns]
order_closed_at_local dtype: datetime64[ns]
business_date dtype: datetime64[ns]


# 4. Handling Missing Values

We will address missing values in key columns. For example, if the restaurant concept is missing in the venues dataset, we will assign it a placeholder. Similarly, if the waiter_uuid is missing in the bills dataset, we will fill it with a placeholder value. We also check for placeholder strings (like "NULL") and convert them to proper NaNs.

In [ ]:
# 1) Fill missing concept with a placeholder
venues_df['concept'] = venues_df['concept'].fillna("UNKNOWN_CONCEPT")

# 2) Fill missing waiter_uuid with a placeholder
bills_df['waiter_uuid'] = bills_df['waiter_uuid'].fillna("UNKNOWN_WAITER")

print("Missing `concept` after fillna:", venues_df['concept'].isnull().sum())
print("Missing `waiter_uuid` after fillna:", bills_df['waiter_uuid'].isnull().sum())

# 3) Check for placeholder strings and convert to NaN
placeholders = ["", "NULL", "NA", "N/A"]
for col in bills_df.columns:
    placeholder_count = bills_df[col].isin(placeholders).sum()
    if placeholder_count > 0:
        bills_df[col].replace(placeholders, np.nan, inplace=True)

# 4) Zero-value checks (for awareness; this is just counting zeros)
numeric_cols = [
    "bill_total_billed",
    "bill_total_discount_item_level",
    "bill_total_gratuity",
    "bill_total_net",
    "bill_total_tax",
    "bill_total_voided",
    "order_duration_seconds",
    "payment_amount",
    "payment_total_tip",
    "sales_revenue_with_tax"
]

zero_counts = {}
for col in numeric_cols:
    zero_counts[col] = (bills_df[col] == 0).sum()

print("\n--- Zero Value Analysis ---")
for col, count in zero_counts.items():
    print(f"{col}: {count} zeros")

Missing `concept` after fillna: 0
Missing `waiter_uuid` after fillna: 0

--- Zero Value Analysis ---
bill_total_billed: 229152 zeros
bill_total_discount_item_level: 8267805 zeros
bill_total_gratuity: 8748366 zeros
bill_total_net: 229618 zeros
bill_total_tax: 1326428 zeros
bill_total_voided: 8715185 zeros
order_duration_seconds: 122 zeros
payment_amount: 230480 zeros
payment_total_tip: 5270130 zeros
sales_revenue_with_tax: 229169 zeros


# 5. Handling Outliers & Negative Values

We remove or adjust certain outliers. For example, order_duration_seconds should not be negative. We also inspect other monetary columns that could be negative. We label rows that appear to be refunds (multiple negative columns) versus normal sales.

In [ ]:
# Remove rows with negative order_duration_seconds
neg_mask = bills_df["order_duration_seconds"] < 0
neg_count = neg_mask.sum()
print(f"Before removal, bills shape: {bills_df.shape}")
if neg_count > 0:
    print(f"Removing {neg_count} rows with negative order_duration_seconds.")
    bills_df = bills_df[~neg_mask].copy()
print(f"After removal, bills shape: {bills_df.shape}")

# Identify negative columns for potential refunds
neg_cols = [
    "bill_total_billed",
    "bill_total_net",
    "bill_total_tax",
    "payment_amount",
    "sales_revenue_with_tax",
    "bill_total_discount_item_level",
    "bill_total_gratuity",
    "bill_total_voided",
    "payment_total_tip"
]

neg_mask_df = bills_df[neg_cols] < 0
bills_df["neg_count"] = neg_mask_df.sum(axis=1)

# Tag refunds if >= 2 negative columns, else sale
bills_df["transaction_type"] = np.where(bills_df["neg_count"] >= 2, "refund", "sale")

refund_count = (bills_df["transaction_type"] == "refund").sum()
sale_count   = (bills_df["transaction_type"] == "sale").sum()

print(f"Refund rows: {refund_count}")
print(f"Sale rows:   {sale_count}")

# Remove the helper column if desired
bills_df.drop(columns=["neg_count"], inplace=True)

Before removal, bills shape: (8954786, 20)
Removing 2 rows with negative order_duration_seconds.
After removal, bills shape: (8954784, 20)
Refund rows: 13844
Sale rows:   8940940


# 6. Merging Datasets

Now we merge the venues_df with the bills_df on the common key venue_xref_id. This combined dataset allows us to analyze both the bill-level and the venue-level attributes together.

In [ ]:
# Merge
merged_df = bills_df.merge(venues_df, on="venue_xref_id", how="left")
print("Merged shape:", merged_df.shape)
print(merged_df.head(3))

# We no longer need bills_df
del bills_df
gc.collect()

Merged shape: (8954784, 25)
   bill_paid_at_local  bill_total_billed  bill_total_discount_item_level  \
0 2024-07-01 09:17:01              21.81                             0.0   
1 2024-07-01 13:37:00              48.95                             0.0   
2 2024-07-01 11:19:32              26.94                             0.0   

   bill_total_gratuity  bill_total_net  bill_total_tax  bill_total_voided  \
0                  0.0       19.299999            2.51                0.0   
1                  0.0       43.320000            5.63                0.0   
2                  0.0       23.840000            3.10                0.0   

                                           bill_uuid business_date  \
0  240701091701~4D63608F-523C-4EFF-9A4F-78D6C44B5159    2024-07-01   
1  240701133659~3A0BEDE2-E9E5-484D-B909-780E485F0D69    2024-07-01   
2  240701111931~670EEB85-E939-4924-B92F-C95076B7E930    2024-07-01   

   order_duration_seconds  ... payment_count payment_total_tip  \
0          

37

# 7. Quick EDA / Outlier Exploration

We can do basic grouping or distribution checks to understand how certain numeric columns look across different concepts or cities. Below is an example grouping.

In [ ]:
summary_cols = ["bill_total_billed", "payment_amount", "payment_total_tip"]

summary = (
    merged_df
    .groupby("concept")[summary_cols]
    .agg(["count", "mean", "std", "min", "max"])
    .reset_index()
)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(summary)

full_merged_df = merged_df.copy() # Create a copy of merged df for later re-merging.

                  concept bill_total_billed                                   \
                                      count        mean         std      min   
0                  BAKERY            306650   15.074988   27.144243  -323.00   
1                     BAR           1255107   38.006514   76.176161 -3323.28   
2                 BREWERY            282848   44.257920  108.833056  -400.00   
3                  BUFFET             19730  102.429040  246.775517     0.00   
4                    CAFE           1454709   20.415212   67.763697  -118.65   
5   ENTERTAINMENT_COMPLEX             52501    5.688213    5.191456   -27.66   
6           FAMILY_DINING           2543870   49.469023  106.147424 -2433.20   
7             FAST_CASUAL            940759   24.262052   36.335240 -4712.10   
8               FAST_FOOD            193537   24.037566   25.000230  -276.02   
9             FINE_DINING            248851   64.281173  106.746725  -696.03   
10             FOOD_TRUCK             19

# 8. Feature Engineering & Modeling: Predicting Unknown Concepts

To illustrate our modeling approach, we will try to predict the concept for venues that have UNKNOWN_CONCEPT. We aggregate bill-level data to the venue level, train a simple Random Forest on venues with known concepts, and then apply that model to venues labeled UNKNOWN_CONCEPT

In [ ]:
needed_cols = [
    "venue_xref_id", "concept",
    "bill_total_billed", "payment_total_tip",
    "order_duration_seconds", "order_take_out_type_label"
]
model_df = merged_df[needed_cols].copy()

# 2) Vectorized tip_percent
model_df["tip_percent"] = model_df["payment_total_tip"] / model_df["bill_total_billed"]
model_df["tip_percent"] = model_df["tip_percent"].fillna(0).replace([np.inf, -np.inf], 0)

# 3) Sample

model_df = model_df.sample(frac=0.95, random_state=42)

# 4) Downcast again
model_df = optimize_numeric(model_df)

# 5) Freed up memory from merged_df

del merged_df
gc.collect()

# Separate known vs unknown concepts
known_df = model_df[model_df["concept"] != "UNKNOWN_CONCEPT"].copy()
unknown_df = model_df[model_df["concept"] == "UNKNOWN_CONCEPT"].copy()

# Aggregate for known data
grp_known = known_df.groupby(["venue_xref_id", "concept"], as_index=False).agg({
    "bill_total_billed": ["mean", "std", "max"],
    "tip_percent": "mean",
    "order_duration_seconds": ["mean", "std"],
    "order_take_out_type_label": lambda x: (x == "takeout").mean()
})

grp_known.columns = [
    "venue_xref_id", "concept",
    "avg_bill", "std_bill", "max_bill",
    "avg_tip_percent",
    "avg_duration", "std_duration",
    "frac_takeout"
]

grp_known.fillna(0, inplace=True)
grp_known = optimize_numeric(grp_known)

# Merge rare classes => "OTHER_CONCEPT"
counts = grp_known["concept"].value_counts()
rare_concepts = counts[counts < 20].index.tolist()

def merge_rare(c):
    return "OTHER_CONCEPT" if c in rare_concepts else c

grp_known["concept"] = grp_known["concept"].apply(merge_rare)

X_all = grp_known.drop(columns=["venue_xref_id", "concept"])
y_all = grp_known["concept"]

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_idx, test_idx in sss.split(X_all, y_all):
    X_train = X_all.iloc[train_idx]
    y_train = y_all.iloc[train_idx]
    X_test = X_all.iloc[test_idx]
    y_test = y_all.iloc[test_idx]

param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [10, 15],
    "class_weight": ["balanced"]
}
clf = RandomForestClassifier(random_state=42)
grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring="f1_weighted",
    cv=3,
    verbose=1
)
grid.fit(X_train, y_train)
best_clf = grid.best_estimator_

print("Best Params:", grid.best_params_)
print("Best CV Score:", grid.best_score_)

y_pred = best_clf.predict(X_test)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))
print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

pred_dict = {}
if not unknown_df.empty:
    grp_unk = unknown_df.groupby("venue_xref_id", as_index=False).agg({
        "bill_total_billed": ["mean", "std", "max"],
        "tip_percent": "mean",
        "order_duration_seconds": ["mean", "std"],
        "order_take_out_type_label": lambda x: (x == "takeout").mean()
    })
    grp_unk.columns = [
        "venue_xref_id",
        "avg_bill", "std_bill", "max_bill",
        "avg_tip_percent",
        "avg_duration", "std_duration",
        "frac_takeout"
    ]
    grp_unk.fillna(0, inplace=True)
    grp_unk = optimize_numeric(grp_unk)

    X_unknown = grp_unk.drop(columns=["venue_xref_id"], errors='ignore')
    if len(X_unknown) > 0:
        unknown_preds = best_clf.predict(X_unknown)
        grp_unk["predicted_concept"] = unknown_preds
        pred_dict = dict(zip(grp_unk["venue_xref_id"], grp_unk["predicted_concept"]))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Params: {'class_weight': 'balanced', 'max_depth': 15, 'n_estimators': 100}
Best CV Score: 0.4156900446491422

=== Classification Report ===
               precision    recall  f1-score   support

          BAR       0.32      0.31      0.31        26
      BREWERY       1.00      0.17      0.29         6
         CAFE       0.33      0.37      0.35        19
FAMILY_DINING       0.55      0.75      0.63        52
  FAST_CASUAL       0.22      0.20      0.21        20
OTHER_CONCEPT       0.00      0.00      0.00        17

     accuracy                           0.42       140
    macro avg       0.40      0.30      0.30       140
 weighted avg       0.38      0.42      0.38       140


=== Confusion Matrix ===
[[ 8  0  2 16  0  0]
 [ 4  1  0  1  0  0]
 [ 2  0  7  3  6  1]
 [ 4  0  3 39  4  2]
 [ 4  0  4  7  4  1]
 [ 3  0  5  5  4  0]]


# 9. Merging Predicted Concepts into the Original Full Dataset

We have created a dictionary `pred_dict` mapping each `venue_xref_id`
to its predicted 'concept'. Now we'll apply this dictionary to
`full_merged_df` to replace all instances of 'UNKNOWN_CONCEPT' with the
newly predicted values. This ensures our original merged dataset
is updated accordingly.


In [ ]:
if pred_dict:
    # Create a new column that looks up the predicted concept for each venue_xref_id
    full_merged_df["updated_concept"] = full_merged_df["venue_xref_id"].map(pred_dict)

    # Where the mapping returns NaN, keep the original concept
    full_merged_df["updated_concept"] = full_merged_df["updated_concept"].fillna(full_merged_df["concept"])

    # Replace old concept column
    full_merged_df["concept"] = full_merged_df["updated_concept"]
    full_merged_df.drop(columns=["updated_concept"], inplace=True)

# Verify the update
print("Final full dataset shape:", full_merged_df.shape)
unknown_remaining = (full_merged_df["concept"] == "UNKNOWN_CONCEPT").sum()
print("Remaining UNKNOWN_CONCEPT entries in full_merged_df:", unknown_remaining)

Final full dataset shape: (8954784, 25)
Remaining UNKNOWN_CONCEPT entries in full_merged_df: 0


In many cases, rows tagged as "refund" can exhibit negative or unusual monetary values
that are not relevant for typical analysis of outliers in standard sales.
Hence, we first labelled rows as "refund" or "sale" based on negative columns.
Then, Now we apply outlier capping exclusively to "sale" rows.

Our concept-based capping function, `concept_based_capping_multiple`,
operates at a group level (by concept) and caps multiple numeric columns
at an upper quantile threshold (e.g., 99th percentile).
This ensures that extremely high values do not overly skew the analysis,
while preserving refund data in its original form.


In [ ]:
import numpy as np
import pandas as pd

def concept_based_capping_multiple_by_group(
    df,
    columns,
    group_col="concept",
    upper_quantile=0.99
):
    # List to hold processed group DataFrames
    df_list = []

    # Iterate over each unique group
    for grp in df[group_col].unique():
        # Select rows for the current group
        group_df = df[df[group_col] == grp].copy()

        # Compute thresholds for the current group for all specified columns
        thresholds = group_df[columns].quantile(upper_quantile)

        # Apply capping: Replace values higher than threshold with the threshold value
        for col in columns:
            group_df[col] = np.minimum(group_df[col], thresholds[col])

        df_list.append(group_df)

    # Concatenate all processed groups into a single DataFrame
    return pd.concat(df_list, ignore_index=True)

# Filter to only "sale" transactions before capping
sale_df = full_merged_df[full_merged_df["transaction_type"] == "sale"].copy()

# List the columns we want to cap
columns_to_cap = [
    "bill_total_billed",
    "bill_total_discount_item_level",
    "bill_total_gratuity",
    "bill_total_net",
    "bill_total_tax",
    "bill_total_voided",
    "order_duration_seconds",
    "payment_amount",
    "payment_total_tip",
    "sales_revenue_with_tax"
]

# Apply the capping function on sale rows
cleaned_sale_df = concept_based_capping_multiple_by_group(
    df=sale_df,
    columns=columns_to_cap,
    group_col="concept",
    upper_quantile=0.99
)

# Explicitly cast each capped column in cleaned_sale_df to the dtype of the corresponding column in full_merged_df
for col in columns_to_cap:
    cleaned_sale_df[col] = cleaned_sale_df[col].astype(full_merged_df[col].dtype)

# Replace the capped values back into the original full DataFrame for sale rows only
full_merged_df.loc[
    full_merged_df["transaction_type"] == "sale",
    columns_to_cap
] = cleaned_sale_df[columns_to_cap].values

print("Capping complete. Full DataFrame shape:", full_merged_df.shape)

# Export the final dataset to a CSV file (without row indices)
full_merged_df.to_csv("final_dataset.csv", index=False)

from google.colab import files
files.download("final_dataset.csv")

Capping complete. Full DataFrame shape: (8954784, 25)


# 10. Visualizing the Effect of Capping

The following plots show the distribution of a representative column
(e.g., "bill_total_billed") for sale transactions before and after outlier capping.The left histogram/boxplot shows the original distribution, while the right one displays the capped values. This is to verify that extreme outliers have been capped without removing any rows.

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

# Choose a representative column to visualize
#column = "bill_total_billed"

# Plot histograms to compare distributions before and after capping
#plt.figure(figsize=(12, 5))

# Histogram before capping
#plt.subplot(1, 2, 1)
#sns.histplot(sale_df[column], bins=50, kde=True, color="blue")
#plt.title(f"Before Capping: {column}")
#plt.xlabel(column)
#plt.ylabel("Frequency")

# Histogram after capping
#plt.subplot(1, 2, 2)
#sns.histplot(cleaned_sale_df[column], bins=50, kde=True, color="green")
#plt.title(f"After Capping: {column}")
#plt.xlabel(column)
#plt.ylabel("Frequency")

#plt.tight_layout()
#plt.show()

# Plot boxplots for additional insight
#plt.figure(figsize=(12, 5))
#data = pd.DataFrame({
#    "Before Capping": sale_df[column],
#    "After Capping": cleaned_sale_df[column]
#})
#sns.boxplot(data=data)
#plt.title(f"Boxplot Comparison for {column}")
#plt.ylabel(column)
#plt.show()

With the 'UNKNOWN_CONCEPT' values now predicted and updated in our
original dataset, we can move on to further analysis or model evaluation.
This pipeline showcases an end-to-end approach:
1. Load & optimize data (downcast for memory efficiency).
2. Merge & handle outliers/missing values.
3. Train a RandomForest model to predict missing concepts.
4. Apply these predicted concepts back to the original data.
5. Capping the outliers at a group level (by concept) and caps multiple numeric columns at an upper quantile threshold (e.g. 99th percentile)

# 11. Analysis

We focus on tip percentage analysis, city-based insights, takeout options, and concept-based distributions, followed by a statistical examination of tipping behavior across different venues.

### 1. Average Tip Percentage by City

In [ ]:
# Mapping cities to countries
city_to_country = full_merged_df.groupby('city')['country'].first().to_dict()

# Calculate tip percentage
full_merged_df['tip_percentage'] = (full_merged_df['payment_total_tip'] / full_merged_df['bill_total_billed']) * 100
full_merged_df['tip_percentage'] = full_merged_df['tip_percentage'].clip(-100, 100)

# Compute average tip percentage by city
average_tip_percentage_by_city = full_merged_df.groupby('city')['tip_percentage'].mean().sort_values(ascending=False)

# Assign colors by country (US = blue, Canada = red)
colors_percentage = ['blue' if city_to_country.get(city, 'US') == 'US' else 'red' for city in average_tip_percentage_by_city.index]

# Plot results
average_tip_percentage_by_city.plot(kind='bar', figsize=(100, 60), color=colors_percentage)
plt.title('Average Tip Percentage by City', fontsize=100)
plt.xlabel('City', fontsize=100)
plt.ylabel('Average Tip Percentage', fontsize=100)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### Insights
- No clear correlation between tipping percentage and country.
- The highest tipping cities include Eads, San Diego, Monson, and Vernon, while the lowest are Bellingham, Greenville, and Richmond Hill.
---

### 2. Top & Bottom Cities by Tip Percentage

In [ ]:
# Display top and bottom 10 cities
print(" Top 10 Cities with Highest Tip Percentage:")
print(average_tip_percentage_by_city.head(10))

print("\n Bottom 10 Cities with Lowest Tip Percentage:")
print(average_tip_percentage_by_city.tail(10))

### Insights

Top 10 Cities

```
Eads            11.35%
San Diego       10.83%
Monson          10.67%
Vernon          10.38%
Scotch Creek    10.24%
Coboconk        10.23%
MESA            10.23%
Statesboro      10.21%
Welland         10.13%
Orange           9.92%
```

Bottom 10 Cities


```
HOUSTON            5.59%
Scaroborogh        5.56%
Richmond Hill      5.51%
Ucluelet           5.42%
Ness City          5.36%
Fredonia           5.27%
Lindenhurst        5.25%
Sault Ste Marie    5.24%
Greenville         5.22%
Bellingham         5.22%
```

Some US cities like Houston show lower tipping percentages, but overall, there’s no clear country-based trend.

---

###3. Takeout Options Distribution for Top & Bottom Cities



In [ ]:
# Extract top and bottom 10 cities' data
top_10_df = full_merged_df[full_merged_df['city'].isin(average_tip_percentage_by_city.head(10).index)]
bottom_10_df = full_merged_df[full_merged_df['city'].isin(average_tip_percentage_by_city.tail(10).index)]

# Takeout options distribution
top_10_takeout = top_10_df['order_take_out_type_label'].value_counts()
bottom_10_takeout = bottom_10_df['order_take_out_type_label'].value_counts()

# Plot pie charts
plt.figure(figsize=(100, 60))
top_10_takeout.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Takeout Options Distribution (Top 10 Tip Cities)')
plt.ylabel('')
plt.show()

plt.figure(figsize=(100, 60))
bottom_10_takeout.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Takeout Options Distribution (Bottom 10 Tip Cities)')
plt.ylabel('')
plt.show()

###Insights
- Top 10 cities are dominated by family dining and bar venues.
- Bottom 10 cities show a more mixed venue distribution, which might be contributing to the lower tipping percentages.

---

### 4. Concept-Based Distribution of Venues

In [ ]:
# Concept type distribution
top_10_concept = top_10_df['concept'].value_counts()
bottom_10_concept = bottom_10_df['concept'].value_counts()

# Plot pie charts
plt.figure(figsize=(100, 60))
top_10_concept.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Concept Distribution (Top 10 Tip Cities)')
plt.ylabel('')
plt.show()

plt.figure(figsize=(100, 60))
bottom_10_concept.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Concept Distribution (Bottom 10 Tip Cities)')
plt.ylabel('')
plt.show()

### Insights
- Top 10 cities primarily have bars and family dining concepts, which tend to encourage tipping.
- Bottom 10 cities have more fast casual and café concepts, where tipping is less common.

---

### 5. Statistical Box Plot Analysis


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Boxplot for Tip Amount by Takeout Option
sns.boxplot(data=full_merged_df, x='order_take_out_type_label', y='payment_total_tip', ax=axes[0])
axes[0].set_title('Box Plot for Tip Amount by Takeout Option')
axes[0].set_xlabel('Takeout Option')
axes[0].set_ylabel('Tip Amount')

# Boxplot for Order Duration by Takeout Option
sns.boxplot(data=full_merged_df, x='order_take_out_type_label', y='order_duration_seconds', ax=axes[1])
axes[1].set_title('Box Plot for Order Duration by Takeout Option')
axes[1].set_xlabel('Takeout Option')
axes[1].set_ylabel('Order Duration (seconds)')

# Boxplot for Billed Amount by Takeout Option
sns.boxplot(data=full_merged_df, x='order_take_out_type_label', y='bill_total_billed', ax=axes[2])
axes[2].set_title('Box Plot for Billed Amount by Takeout Option')
axes[2].set_xlabel('Takeout Option')
axes[2].set_ylabel('Billed Amount')

plt.tight_layout()
plt.show()

## Insights
- Tipping varies significantly across takeout options, with dine-in and bar tabs showing the most variation.
- Order durations and billed amounts have several outliers, indicating large transactions.

---

### 6. Average Tip Amount by City

In [ ]:
# Compute average tip amount by city
average_tip_amt_by_city = full_merged_df.groupby('city')['payment_total_tip'].mean().sort_values(ascending=False)

# Assign colors by country
colors_amt = ['blue' if city_to_country.get(city, 'US') == 'US' else 'red' for city in average_tip_amt_by_city.index]

# Plot results
average_tip_amt_by_city.plot(kind='bar', figsize=(100, 60), color=colors_amt)
plt.title('Average Tip Amount by City', fontsize=100)
plt.xlabel('City', fontsize=100)
plt.ylabel('Average Tip Amount', fontsize=100)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Insights
- Worth Township, Eads, and Woodstock lead in absolute tip amount, suggesting high-value bills in these cities.

---

# Overall Analysis
---

## 1. **Objective & Key Questions**

**Objective (#8)**  
> Investigate how tipping behaviors vary across different geographic locations. Compare average tip amounts or tip percentages (tip ÷ billed amount) by city and country to see if there are notable regional differences. Look for correlations with venue attributes, order types, and other factors that might influence tipping culture.

From this, we posed several guiding questions:

1. **Which cities have the highest and lowest average tip percentages?**  
2. **Do these tipping differences correlate with the venue’s country (US vs. Canada)?**  
3. **How do concept types (e.g., BAR, CAFE, FINE_DINING) and order types (dinein, takeout, delivery, etc.) influence tipping?**  
4. **Do high tip percentages also coincide with high absolute tip amounts?**  

---

## 2. **Data Preparation Overview**

Before focusing on tipping, we addressed the following:

- **Merging Datasets**: Joined `bills.csv` and `venues.csv` on `venue_xref_id`, yielding a unified DataFrame (`full_merged_df`).  
- **Outlier Handling**:  
  - Removed rows with negative `order_duration_seconds`.  
  - Tagged bills with multiple negative monetary columns as “refund” vs. “sale.”  
  - Applied capping (99th percentile) by concept group for “sale” transactions, preventing extreme outliers from skewing results.  
- **Missing Values**:  
  - Filled missing `concept` with either a predicted value (via RandomForest) or a placeholder (`UNKNOWN_CONCEPT` initially).  
  - Filled missing `waiter_uuid` with a placeholder (`UNKNOWN_WAITER`).  

**Tip Percentage Calculation**  
We created a `tip_percentage` column:  
$$
\text{tip\_percentage} = \frac{\text{payment\_total\_tip}}{\text{bill\_total\_billed}} \times 100
$$
Values were clipped to \([-100\%, +100\%]\) to remove any more erroneous outliers.

---

## 3. **High-Level Findings**

1. **Large Variation in Tip Percentages Across Cities**  
   - Some cities (e.g., *Eads*, *San Diego*, *Monson*) average over 10% in tip percentage.  
   - Others (e.g., *Bellingham*, *Greenville*, *Richmond Hill*) hover near 5%.  

2. **No Strong “Country-Level” Trend**  
   - Contrary to a simple hypothesis that the US might tip more than Canada (or vice versa), we see both high and low tip-percentage cities in each country. The color-coded bar charts (blue = US, red = Canada) show no consistent pattern purely by national boundary.

3. **Concept Type Influences Tipping**  
   - Bars and family dining concepts often see higher tip percentages than cafés and fast casual venues.  
   - Pie charts for the top 10 vs. bottom 10 tipping cities suggest that top-tipping locales are dominated by *BAR* and *FAMILY_DINING* concepts, whereas lower-tipping locales have more *CAFE* or *FAST_CASUAL*.

4. **Order Type Matters**  
   - Box plots indicate that **dine-in** and **bar tab** transactions yield higher tip amounts (and higher variation) than **takeout** or **delivery** orders.  

5. **High Absolute Tip Amounts vs. High Tip Percentages**  
   - Certain cities (e.g., *Worth Township*, *Eads*, *Woodstock*) appear at the top of the **Average Tip Amount** bar chart. This sometimes coincides with high bills, so they tip more in total dollars but not always the highest in percentage.  

---

## 4. **Detailed Visual Analysis**

### 4.1. **Average Tip Percentage by City**

- **Left side (High Tippers)**: *Eads (11.35%)*, *San Diego (10.83%)*, *Monson (10.67%)*, *Vernon (10.38%)*, *Scotch Creek (10.24%)*, *Coboconk (10.23%)*  
- **Right side (Low Tippers)**: *Bellingham (5.22%)*, *Greenville (5.22%)*, *Sault Ste Marie (5.24%)*, *Richmond Hill (5.51%)*, *Scaroborogh (5.56%)*

Despite color-coding by country (blue = US, red = Canada), high- and low-tipping cities are found in both countries. This suggests local city culture, venue mix, or other factors play a larger role than simple national boundaries.

---

### 4.2. **Top 10 vs. Bottom 10 Cities: Takeout Options**

- In the **top 10** tipping cities, we see a large portion of **dinein** or **bar tab** orders.  
- In the **bottom 10**, the distribution is more mixed: more **takeout**, **delivery**, or **onlineorder**. Tipping is typically lower in these formats.

---

### 4.3. **Top 10 vs. Bottom 10 Cities: Concept Distribution**

- **Top 10** are heavily dominated by *BAR*, *FAMILY_DINING*, or similar.  
- **Bottom 10** shows a bigger slice for *CAFE*, *FAST_CASUAL*, or smaller niche concepts.  

It aligns with the common pattern that bars and full-service dining see higher tipping rates.

---

### 4.4. **Box Plots by Takeout Option**

1. **Tip Amount**: *bar tab* and *dinein* exhibit the largest spread in tip amounts. *onlineorder*, *takeout*, and *delivery* generally cluster lower.  
2. **Order Duration**: Dine-in can have longer durations than quick-service or takeout orders.  
3. **Billed Amount**: Bar tabs and dine-in often have higher bills, which can drive higher absolute tips (but not always the highest tip percentage).

---

### 4.5. **Average Tip Amount by City**

- Cities like *Worth Township*, *Eads*, and *Woodstock* lead in absolute tip amounts. This usually indicates larger bills (e.g., premium dining or large-group tabs).  
- Some high-dollar tips coincide with high tip percentages (e.g., Eads), but not always—some cities might tip big in dollar terms simply because their bills are large.

---

## 5. **Correlations & Interpretations**

1. **City vs. Country**  
   - The data does **not** reveal a straightforward “US always tips more” or “Canada always tips more” conclusion. Instead, each country has pockets of high- and low-tipping behavior.  
   - Localized venue mix (bars vs. cafés) and cultural norms in certain cities seem more decisive than national averages.

2. **Concept Matters Greatly**  
   - Bars, breweries, and full-service restaurants (often labeled *FAMILY_DINING* or *FINE_DINING*) encourage tipping culture more strongly.  
   - Cafés, fast casual, or quick-service restaurants see lower tip percentages. The environment and ordering style (counter service vs. table service) can reduce tipping.

3. **Order Type Impacts Tips**  
   - Dine-in experiences have higher tip amounts/percentages. *Takeout* and *delivery*—especially in North America—tend to yield lower tips (or none), consistent with broader industry trends.

4. **Bill Size & Absolute Tip**  
   - Larger checks lead to higher absolute tips. However, tip percentage can remain moderate if the bill is very high. This is why some cities rank high in average tip amount but not necessarily in tip percentage.

5. **Outliers & Data Quality**  
   - After capping extreme values, the overall distribution remains similar. A few extreme transactions can skew the average tip amount or tip percentage, so analyzing median or trimming outliers can give a clearer view.

---

## 6. **Practical Recommendations**

1. **Venue Positioning**  
   - **Bars & Full-Service Restaurants** in both countries can emphasize the tipping aspect by training staff to provide exceptional table service. This correlates with higher tips.  
   - **Cafés & Fast Casual** may consider alternative tipping strategies (e.g., digital tip prompts) if they want to increase tip capture.

2. **Regional Marketing**  
   - For *low-tipping* cities, highlight “tip suggestion” lines or adopt recommended gratuity guidelines on the bill to gently nudge customers.  
   - For *high-tipping* cities, reinforcing the existing tip culture can further boost staff morale and revenue.

3. **Optimize Order Types**  
   - Since *takeout* and *delivery* see lower tips, restaurants reliant on off-premise orders might explore small “service fees” or “optional gratuity” prompts to maintain staff wage support.

4. **Refine Staffing & Pricing**  
   - In high-tip-percentage cities, staff are often more motivated by tip earnings; scheduling more servers during peak times could lead to better guest experiences and still net strong tip amounts.  
   - In low-tip regions or concepts, focus on operational efficiency and possibly higher wages if tips are historically low.

5. **Future Data Enhancements**  
   - Incorporate local demographics or economic indicators to see if cost of living or median income correlates with tipping patterns.  
   - Compare *weekend vs. weekday* or *holiday vs. non-holiday* to see if there are temporal tip differences across the same cities.

---

## 7. **Conclusion**

In sum, **tipping culture is shaped more by venue concept and order type than by country alone.** While some US cities top the list in tip percentage and absolute tip amount, certain Canadian cities also appear among the highest (and lowest) tippers. The strongest consistent factor is whether the transaction is a *bar tab/dine-in* (tends to yield higher tips) or *takeout/delivery* (lower tips). Moreover, concepts like *BAR* and *FAMILY_DINING* encourage tip culture more than *CAFE* or *FAST_CASUAL*.

For restaurateurs, **knowing the local tipping norms** and aligning service style can help optimize revenues and staff satisfaction. By customizing tip strategies (e.g., digital tip prompts for takeout, suggested gratuities for dine-in), restaurants can better navigate the complexities of regional tipping behaviors.

---

### **Key Takeaways**

- **Highest Tip Percentages**: Eads, San Diego, Monson, Vernon, Scotch Creek  
- **Lowest Tip Percentages**: Bellingham, Greenville, Sault Ste Marie, Richmond Hill, Scaroborogh  
- **Bars & Full-Service**: Higher tip culture  
- **Takeout & Delivery**: Lower tips, consistent with quick-service norms  
- **Country Factor**: Not definitive; local city culture and concept type overshadow a simple US vs. Canada split  
